In [1]:
import pandas as pd
import numpy as np

In [2]:
# np.arange : 특정 수열 생성
X_value = np.arange(40).reshape(20, 2)
y_value = np.arange(20)

In [3]:
# 검증하기 위한 데이터프레임 생성
sample_df = pd.DataFrame(np.column_stack((X_value, y_value)), columns = ['X_1', 'X_2',
                                                    'result'])

# 생성된 데이터 확인
print(sample_df.shape)

sample_df.head(3)

(20, 3)


,X_1,X_2,result
0,0,1,0
1,2,3,1
2,4,5,2


In [4]:
# 랜덤표본추출의 다양한 방법들

# n : 표본 추출 개수
# random_state : 난수 발생 초기값(재현을 위해 사용)
sample_df.sample(n=5, random_state = 1001)

,X_1,X_2,result
1,2,3,1
15,30,31,15
0,0,1,0
2,4,5,2
18,36,37,18


In [5]:
# DataFrame 으로부터 특정 비율만큼 표본 추출
# frac : 추출할 표본 비율
sample_df.sample(frac = 0.5, random_state = 1001)

# n 대신에 frac 을 이용하면 전체 데이터에서 몇 %를 뽑아올 지를 정의할 수 있다.

,X_1,X_2,result
1,2,3,1
15,30,31,15
0,0,1,0
2,4,5,2
18,36,37,18
7,14,15,7
10,20,21,10
6,12,13,6
19,38,39,19
4,8,9,4


In [6]:
# DataFrame 으로부터 복원 무작위 표본 추출

rep_df = sample_df.sample(frac = 0.2, random_state = 1001) # 샘플 개수 20%인 4개만 가져옴
print(rep_df.shape)
rep_df.head()

(4, 3)


,X_1,X_2,result
1,2,3,1
15,30,31,15
0,0,1,0
2,4,5,2


In [7]:
# replace = True : 복원추출
# rep_df 4개 샘플읇 복원추출로 10개로 만들기
rep_df.sample(n=10, replace = True, random_state = 1001)

,X_1,X_2,result
15,30,31,15
15,30,31,15
15,30,31,15
0,0,1,0
0,0,1,0
2,4,5,2
1,2,3,1
1,2,3,1
0,0,1,0
2,4,5,2


In [8]:
## DataFrame 내의 특정 칼럼의 값을 기준으로 가중치를 부여하여 무작위 표본 추출
# wieghts : 가중치를 반영할 필드값
sample_df.sample(n=5, weights = 'result') # 여기서는 result 가 가중치를 반영할 필드값이다.

,X_1,X_2,result
10,20,21,10
11,22,23,11
18,36,37,18
3,6,7,3
4,8,9,4


In [9]:
# 계통 표본 추출 함수 정의
def sysmetic_sampling(data, n) :
    count = len(data) # 모집단수
    sample_count = count // n
    index = data[:sample_count].sample(1).index
    intoin = index - 0 #샘플 간 간격
    sys_df = pd.DataFrame()
    while len(sys_df) < n :
        sys_df = sys_df.append(data.loc[index, :])
        index += sample_count
    return(sys_df)

In [10]:
# 구간 내 필요 샘플 수로 간격 정의
# 함수 호출
sysmetic_sampling(sample_df, 5)

C:\Users\user\AppData\Local\Temp\ipykernel_2356\1877885624.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sys_df = sys_df.append(data.loc[index, :])
C:\Users\user\AppData\Local\Temp\ipykernel_2356\1877885624.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sys_df = sys_df.append(data.loc[index, :])
C:\Users\user\AppData\Local\Temp\ipykernel_2356\1877885624.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sys_df = sys_df.append(data.loc[index, :])
C:\Users\user\AppData\Local\Temp\ipykernel_2356\1877885624.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sys_df = sys_df.append(data.loc[index, :])
C:\Users\user\AppData\Local\

,X_1,X_2,result
0,0,1,0
4,8,9,4
8,16,17,8
12,24,25,12
16,32,33,16


In [11]:
# 층화확률표본추출
# 패키지 로드 및 설정
from sklearn.model_selection import StratifiedShuffleSplit

# StratifiedShuffleSplit (n_splits : 분할반복횟수, test_size : 테스트셋  샘플 비율)

splitfi = StratifiedShuffleSplit(n_splits=1, test_size = 0.3, random_state = 1001)

In [12]:
# 현재 데이터 확인
sample_df.head(3)

,X_1,X_2,result
0,0,1,0
1,2,3,1
2,4,5,2


In [13]:
# group 값 없음

In [14]:
# 층 구별을 위한 값 (0: 0그룹, 1:1그룹)
group = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
sample_df['group'] = group

In [15]:
# 전체 데이터 수 확인
sample_df["group"].value_counts()

0    10
1    10
Name: group, dtype: int64

In [16]:
# df_strat_train, df_strat_test 으로 데이터 분할(표본 추출)
for train_idx, test_idx in splitfi.split(sample_df, sample_df['group']) :
    print("Train :", train_idx, "Test:", test_idx)
    df_strat_train = sample_df.loc[train_idx]
    df_strat_test = sample_df.loc[test_idx]

Train : [13  7  1 14 16 12  0 11 10 18  2  8  5  6] Test: [17 19  3 15  4  9]


In [17]:
print("Train data 수 확인")
print(df_strat_train.shape)
print("Test data 수 확인")
print(df_strat_test.shape)

Train data 수 확인
(14, 4)
Test data 수 확인
(6, 4)


In [18]:
# 모집단과 동일 비율로 Group 속성을 기준으로 데이터 분리 확인
print("전체 비율")
print(sample_df['group'].value_counts() / len(sample_df))
print("Train data 비율")
print(df_strat_train['group'].value_counts() / len(df_strat_train))
print(df_strat_test['group'].value_counts() / len(df_strat_test))

전체 비율
0    0.5
1    0.5
Name: group, dtype: float64
Train data 비율
1    0.5
0    0.5
Name: group, dtype: float64
1    0.5
0    0.5
Name: group, dtype: float64
